In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%cd ..

In [1]:
from deep_translator import GoogleTranslator
import json
import re
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
mixed_prompts = None
with open('./datasets/yujie/prompts/director_prompts_Midjourney.json', "r", encoding="utf-8") as f:
    mixed_prompts = json.load(f)
mixed_prompts[0:5]

In [ ]:
mixed_prompts[53]

In [6]:
def process_item(i, item):
    # Function to process each item in parallel
    translator = GoogleTranslator()
    if isinstance(item, str):
        curr_mixed = item
        chinese_words = re.findall(r'[\u4e00-\u9fff]+', curr_mixed)
        for word in chinese_words:
            translated = translator.translate(word)
            # print(word,"->", translated)
            curr_mixed = curr_mixed.replace(word, ' ' + translated + ' ')
        return i, curr_mixed

    elif isinstance(item, dict):
        if 'prompt' in item:
            curr_mixed = item['prompt']
            chinese_words = re.findall(r'[\u4e00-\u9fff]+', curr_mixed)
            for word in chinese_words:
                translated = translator.translate(word)
                # print(word,"->", translated)
                curr_mixed = curr_mixed.replace(word, ' ' + translated + ' ')
        return i, curr_mixed

    else:
        raise TypeError("Item must be either a string or a dictionary with 'prompt'")


In [ ]:
full_eng_prompts = [None] * len(mixed_prompts)

# Multithreading for parallel processing
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_item, i, item): i for i, item in enumerate(mixed_prompts)}
    
    for future in as_completed(futures):
        i, result = future.result()
        full_eng_prompts[i] = result
        
        i = futures[future]  # Retrieve the original index
        if i % 10 == 0:
            with open("./datasets/yujie/prompts/eng_director_prompts_Midjourney.json", "w", encoding="utf-8") as f:
                json.dump(full_eng_prompts, f, ensure_ascii=False, indent=4)
            print(i)
full_eng_prompts

In [8]:
with open("./datasets/yujie/prompts/eng_director_prompts_Midjourney.json", "w", encoding="utf-8") as f:
    json.dump(full_eng_prompts, f, ensure_ascii=False, indent=4)

In [9]:
# full_eng_prompts = []
# i = 0
# while i < len(mixed_prompts):
#     item = mixed_prompts[i]
#     result = None
#     if type(item) == str:
#         curr_mixed = item
#         chinese_words = re.findall(r'[\u4e00-\u9fff]+', curr_mixed)
#         for word in chinese_words:
#             translated = translator.translate(word)
#             curr_mixed = curr_mixed.replace(word, ' ' + translated + ' ')

#         result = curr_mixed
#     elif type(item) == dict:
#         if 'prompt' in item:
#             curr_mixed = item['prompt']
#             chinese_words = re.findall(r'[\u4e00-\u9fff]+', curr_mixed)
#             for word in chinese_words:
#                 translated = translator.translate(word)
#                 curr_mixed = curr_mixed.replace(word,' ' + translated + ' ')

#             result = curr_mixed

#         else:
#             raise TypeError
#     else:
#         raise TypeError
    
#     full_eng_prompts.append(result)
#     if i % 10 == 0:
#         print(i, result)
#     i += 1
#     # break
# full_eng_prompts